want to compare the increase of cabi rides between maryland and dc

2022,2023,2024

steps
filter the data set for stations and rides in dc and maryland:
1) take the complete dataset, turn it into geodataframe
2) join it with maryland gdf
3) make plot to compare the difference of growth between them

dive deeper into maryland / prince george county for 2023 -2024

In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
from geopy import distance
import folium
from folium.plugins import MarkerCluster
from folium.features import GeoJsonTooltip
from branca.colormap import LinearColormap
from collections import Counter
import json
from shapely.geometry import Point
import geopandas as gpd
from shapely.geometry import shape

In [2]:
data_types = {
    "rideable_type": "category", 
    "start_station_name": "category", 
    "end_station_name": "category", 
    "member_casual":"category",
    # "ride_id":"uint32",
    "time_of_day":"category",
    "trip_type":"category"}

In [3]:
bikes = pd.read_csv("bikes_clean.csv",parse_dates= ["started_at", "ended_at"], dtype=data_types,low_memory=False)

In [4]:
cabi_2024 = pd.read_csv("cabi_2024.csv",parse_dates= ["started_at", "ended_at"], low_memory=False)

In [7]:
cabi_updated = pd.concat([bikes, cabi_2024],ignore_index=True,axis=0)

In [ ]:
cabi_updated.shape

(16808320, 20)

In [8]:
cabi_updated =cabi_updated.drop(columns = ["ride_id","start_station_id","end_station_id","start_time","end_time","start_hour","end_hour","Unnamed: 0"])

In [9]:
cabi_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16808320 entries, 0 to 16808319
Data columns (total 12 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   rideable_type          object  
 1   started_at             object  
 2   ended_at               object  
 3   start_station_name     object  
 4   end_station_name       object  
 5   member_casual          object  
 6   start_lat              float64 
 7   start_lng              float64 
 8   end_lat                float64 
 9   end_lng                float64 
 10  trip_duration_minutes  float64 
 11  time_of_day            category
dtypes: category(1), float64(5), object(6)
memory usage: 1.4+ GB


In [10]:
cabi_updated["rideable_type"] = cabi_updated["rideable_type"].astype("category")
cabi_updated["start_station_name"] = cabi_updated["start_station_name"].astype("category")
cabi_updated["end_station_name"] = cabi_updated["end_station_name"].astype("category")
cabi_updated["member_casual"] = cabi_updated["member_casual"].astype("category")


In [14]:
cabi_updated['started_at'] = pd.to_datetime(cabi_updated['started_at'])
cabi_updated['year'] = cabi_updated['started_at'].dt.year

In [17]:
# selecting only 2022,2023,2024
cabi_updated_2022_to_2024 = cabi_updated[(cabi_updated["year"] == 2022)| (cabi_updated["year"] == 2023)|(cabi_updated["year"] == 2024)]

In [18]:
cabi_updated_2022_to_2024['started_at'] = cabi_updated_2022_to_2024['started_at'].astype("object")

In [19]:
cabi_updated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14058439 entries, 2749881 to 16808319
Data columns (total 13 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   rideable_type          category
 1   started_at             object  
 2   ended_at               object  
 3   start_station_name     category
 4   end_station_name       category
 5   member_casual          category
 6   start_lat              float64 
 7   start_lng              float64 
 8   end_lat                float64 
 9   end_lng                float64 
 10  trip_duration_minutes  float64 
 11  time_of_day            category
 12  year                   int32   
dtypes: category(5), float64(5), int32(1), object(2)
memory usage: 1005.6+ MB


In [20]:
# # Create a GeoDataFrame for the start stations
geometry = [Point(xy) for xy in zip(cabi_updated['start_lng'], cabi_updated['start_lat'])]
cabi_updated_gdf = gpd.GeoDataFrame(cabi_updated, geometry=geometry, crs="EPSG:4326")



In [21]:
cabi_updated_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 14058439 entries, 2749881 to 16808319
Data columns (total 14 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   rideable_type          category
 1   started_at             object  
 2   ended_at               object  
 3   start_station_name     category
 4   end_station_name       category
 5   member_casual          category
 6   start_lat              float64 
 7   start_lng              float64 
 8   end_lat                float64 
 9   end_lng                float64 
 10  trip_duration_minutes  float64 
 11  time_of_day            category
 12  year                   int32   
 13  geometry               geometry
dtypes: category(5), float64(5), geometry(1), int32(1), object(2)
memory usage: 1.1+ GB


In [ ]:
# cabi_updated_gdf['started_at'] = pd.to_datetime(cabi_updated_gdf['started_at'])#, format='%Y-%m-%d %H:%M:%S')
# cabi_updated_gdf['ended_at'] = pd.to_datetime(cabi_updated_gdf['ended_at'])
# cabi_updated_gdf['year'] = cabi_updated_gdf['started_at'].dt.year
# cabi_updated_gdf['month'] = cabi_updated_gdf['started_at'].dt.month

In [ ]:
# cabi_updated_gdf.to_csv("cabi_updated_gdf.csv")

# sample

In [ ]:
# # cabi updated gdf sample 

# # 108.000 rows sample (3000 per month)
# samples = []

# # Loop over years and months to sample 1,000 rows for each month
# for year in range(2021, 2025):  # Covering 2021, 2022, and 2024
#     for month in range(1, 13):  # Months from 1 to 12
#         # Filter for the specific year and month
#         filtered_data = cabi_updated_gdf[(cabi_updated_gdf["year"] == year) & (cabi_updated_gdf["month"] == month)]
        
#         # Sample 1,000 rows if there are enough rows available
#         if len(filtered_data) >= 3000:
#             sample = filtered_data.sample(n=3000, random_state=42)
#         else:  # If less than 3,000 rows, take all available rows
#             sample = filtered_data
            
#         samples.append(sample)

# # Combine all the samples into a single DataFrame
# bikes_monthly_sample_3000 = pd.concat(samples, ignore_index=True)

In [ ]:
# bikes_monthly_sample_3000.to_csv("bikes_sample_updated.csv", index=False)

In [22]:
# uploading ward info so I can filter out DC area
with open ("Wards_from_2022.geojson") as i:
    wards = json.loads(i.read())

features =wards["features"]

ward_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326" )

In [23]:
ward_gdf = ward_gdf[["WARD","NAME","geometry"]]

In [24]:
dc_wards_gdf = gpd.sjoin(cabi_updated_gdf, ward_gdf, how="left", predicate="within")

In [25]:
with open ("Maryland_Physical_Boundaries_-_County_Boundaries_(Detailed).geojson") as i:
    maryland = json.loads(i.read())

features = maryland["features"]

#GDF
maryland_gdf = gpd.GeoDataFrame(
    pd.DataFrame([feature['properties'] for feature in features]),  # Extract properties as attributes
    geometry=[shape(feature['geometry']) for feature in features],  # Convert geometries
    crs="EPSG:4326")
    

In [ ]:
maryland_gdf

In [26]:
dc_wards_gdf = dc_wards_gdf.drop(columns=["index_right"])

In [36]:
dc_wards_gdf

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,start_lat,start_lng,end_lat,end_lng,trip_duration_minutes,time_of_day,year,geometry,WARD,NAME
2749881,classic_bike,2022-01-01 00:04:52,2022-01-01 00:10:40,Thomas Circle,17th & K St NW,casual,38.905900,-77.032500,38.902760,-77.038630,6.0,night,2022,POINT (-77.0325 38.9059),2.0,Ward 2
2749882,electric_bike,2022-01-01 00:05:14,2022-01-01 00:15:39,14th & R St NW,NaN,casual,38.913055,-77.031823,38.930000,-77.050000,10.0,night,2022,POINT (-77.03182 38.91306),2.0,Ward 2
2749883,classic_bike,2022-01-01 00:06:00,2022-01-01 00:20:47,4th St & Madison Dr NW,10th & K St NW,casual,38.890496,-77.017246,38.902400,-77.026220,15.0,night,2022,POINT (-77.01725 38.8905),2.0,Ward 2
2749884,classic_bike,2022-01-01 00:06:01,2022-01-01 00:20:39,4th St & Madison Dr NW,10th & K St NW,casual,38.890496,-77.017246,38.902400,-77.026220,15.0,night,2022,POINT (-77.01725 38.8905),2.0,Ward 2
2749885,classic_bike,2022-01-01 00:06:19,2022-01-01 00:12:36,14th & Newton St NW,14th & V St NW,member,38.931991,-77.032956,38.917764,-77.032096,6.0,night,2022,POINT (-77.03296 38.93199),1.0,Ward 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16808315,electric_bike,2024-12-21 17:51:11.338000,2024-12-21 17:54:36.282,11th & M St NW,NaN,member,38.905578,-77.027313,38.910000,-77.020000,NaN,NaN,2024,POINT (-77.02731 38.90558),2.0,Ward 2
16808316,electric_bike,2024-12-07 10:44:26.102000,2024-12-07 10:52:42.706,11th & M St NW,14th & Belmont St NW,member,38.905578,-77.027313,38.921074,-77.031887,NaN,NaN,2024,POINT (-77.02731 38.90558),2.0,Ward 2
16808317,electric_bike,2024-12-08 19:09:28.769000,2024-12-08 19:12:37.130,11th & M St NW,NaN,member,38.905578,-77.027313,38.900000,-77.030000,NaN,NaN,2024,POINT (-77.02731 38.90558),2.0,Ward 2
16808318,electric_bike,2024-12-26 11:52:51.207000,2024-12-26 11:57:38.459,11th & M St NW,NaN,member,38.905578,-77.027313,38.900000,-77.030000,NaN,NaN,2024,POINT (-77.02731 38.90558),2.0,Ward 2


In [27]:
maryland_gdf=maryland_gdf[["COUNTY","geometry"]]

In [28]:
# # Perform spatial join with the maryland gdf
dc_maryland_gdf= gpd.sjoin(dc_wards_gdf, maryland_gdf, how="left", predicate="within")

In [29]:
dc_maryland_gdf[["NAME","COUNTY"]]

,NAME,COUNTY
2749881,Ward 2,NaN
2749882,Ward 2,NaN
2749883,Ward 2,NaN
2749884,Ward 2,NaN
2749885,Ward 1,NaN
...,...,...
16808315,Ward 2,NaN
16808316,Ward 2,NaN
16808317,Ward 2,NaN
16808318,Ward 2,NaN


In [32]:
# filtering out values that are not in maryland or dc
dc_maryland_gdf = dc_maryland_gdf[dc_maryland_gdf["NAME"].notnull()| dc_maryland_gdf["COUNTY"].notnull()]

In [33]:
# new column to determine easy if it's dc or maryland
dc_maryland_gdf['area'] = np.where(dc_maryland_gdf['WARD'].notnull(), 'DC', 'Maryland')

/opt/anaconda3/envs/codeacademy/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [38]:
df_2022 = dc_maryland_gdf[dc_maryland_gdf["year"]==2022]
df_2023 = dc_maryland_gdf[dc_maryland_gdf["year"]==2023]
df_2024 = dc_maryland_gdf[dc_maryland_gdf["year"]==2024]

In [ ]:
# dc_maryland_gdf.to_csv("dc_maryland_gdf_full_updated.csv", index=False)

In [1]:
#Total rides per year
print("total rides in 2022", len(df_2022))
print("total rides in 2023", len(df_2023))
print("total rides in 2024", len(df_2024))


NameError: name 'df_2022' is not defined

In [ ]:
# Rides per year per area
rides_DC = dc_maryland_gdf[dc_maryland_gdf["area"]=="DC"]
rides_maryland = dc_maryland_gdf[dc_maryland_gdf["area"]=="Maryland"]

: 

In [48]:
#number of unique stations per year
print("unique stations in 2022", df_2022["start_station_name"].nunique())
print("unique stations in 2023", df_2023["start_station_name"].nunique())
print("unique stations in 2022", df_2024["start_station_name"].nunique())

unique stations in 2022 490
unique stations in 2023 564
unique stations in 2022 596
